In [1]:
import requests as requests
import pandas as pd
import numpy as np
import json


# Frost

In [2]:
client_id = 'a92d628d-b382-48b8-b157-b60da4ae9dc6'
endpoint = 'https://frost.met.no/observations/v0.jsonld'

time_frame = '2019-01-01/2021-07-18'
paramPrec = {
    'sources': 'SN18700',
    'elements': 'sum(precipitation_amount P1D)',
    'referencetime': time_frame,

}

paramTemp = {
    'sources': 'SN18700',
    'elements': 'air_temperature',
    'referencetime': time_frame,
    "timeresolutions":'PT1H',
}

paramCloud = {
    'sources': 'SN18700',
    'elements': 'mean(cloud_area_fraction P1D)',
    'referencetime': time_frame,

}

paramHum = {
    'sources': 'SN18700',
    'elements': 'relative_humidity',
    'referencetime': time_frame,
    "timeresolutions":'PT1H',
}

paramWindDir = {
    'sources': 'SN18700',
    'elements': 'wind_from_direction',
    'referencetime': time_frame,
    "timeresolutions":'PT1H'
}

paramWindSpeed = {
    'sources': 'SN18700',
    'elements': 'wind_speed',
    'referencetime': time_frame,
    "timeresolutions":'PT1H'
}
paramPressure= {
    'sources': 'SN18700',
    'elements': 'air_pressure_at_sea_level',
    'referencetime': time_frame,
    "timeresolutions":'PT1H'
}


#'air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed',
r = requests.get(endpoint,params=paramPrec, auth=(client_id,''))
jsonPrec = r.json()

r = requests.get(endpoint,params=paramTemp, auth=(client_id,''))
jsonTemp = r.json()

r = requests.get(endpoint,params=paramCloud, auth=(client_id,''))
jsonCloud = r.json()

r = requests.get(endpoint,params=paramHum, auth=(client_id,''))
jsonHum = r.json()

r = requests.get(endpoint,params=paramWindDir, auth=(client_id,''))
jsonWindDir = r.json()

r = requests.get(endpoint,params=paramWindSpeed, auth=(client_id,''))
jsonWindSpeed = r.json()

r = requests.get(endpoint,params=paramPressure, auth=(client_id,''))
jsonPressure = r.json()


In [3]:
dataTypes = [jsonPrec,jsonTemp,jsonCloud,jsonHum,jsonWindDir,jsonWindSpeed,jsonPressure]

#requests.get('https://frost.met.no//observations/availableTimeSeries',auth=(client_id,''))

# OBS! Denne tar typ 10 min å kjøre!

In [11]:
# This will return a Dataframe with all of the observations in a table format
from tqdm import tqdm
dfFrost = pd.DataFrame()
for d in tqdm(dataTypes):
    data=d['data']
    for i in range(len(data)):
        row = {}
        #row=row.drop(["timeOffset","timeResolution","timeSeriesId"],axis=1)
        row['value'] = [data[i]['observations'][0]['value']]
        row['referenceTime'] = [data[i]['referenceTime']]
        row['sourceId'] = [data[i]['sourceId']]
        row['elementId'] = [data[i]['observations'][0]['elementId']]
        row = pd.DataFrame.from_dict(row)
        dfFrost = dfFrost.append(row)
   
        
dfFrost = dfFrost.reset_index(drop=True)
#dfFrost.referenceTime=np.array(df['referenceTime'],dtype="datetime64")


100%|████████████████████████████████████████████| 7/7 [14:12<00:00, 121.79s/it]


In [50]:
#dfFrost = pd.read_csv('frostAggregertTime.csv',sep=';',low_memory=False)
dfFrost["referenceTime"]=[str(v)[:-10] for v in np.array(dfFrost["referenceTime"],dtype="datetime64")]

dfFrost['referenceTime']
dfFrost

,value,referenceTime,sourceId,elementId
0,0.0,2019-01-01T00:00:00,SN18700:0,sum(precipitation_amount P1D)
1,0.0,2019-01-02T00:00:00,SN18700:0,sum(precipitation_amount P1D)
2,0.0,2019-01-03T00:00:00,SN18700:0,sum(precipitation_amount P1D)
3,0.0,2019-01-04T00:00:00,SN18700:0,sum(precipitation_amount P1D)
4,0.0,2019-01-05T00:00:00,SN18700:0,sum(precipitation_amount P1D)
...,...,...,...,...
113331,1011.5,2021-07-17T19:00:00,SN18700:0,air_pressure_at_sea_level
113332,1011.1,2021-07-17T20:00:00,SN18700:0,air_pressure_at_sea_level
113333,1010.9,2021-07-17T21:00:00,SN18700:0,air_pressure_at_sea_level
113334,1011.0,2021-07-17T22:00:00,SN18700:0,air_pressure_at_sea_level


In [51]:
dfFrost.to_csv("data/frostAggregertTime.csv",sep=';',index=False)

In [31]:
dfFrost['referenceTime'][0]

'2019-01-01T00:00:00.000000000'

# Forecast


In [33]:
# Define endpoint and parameters

endpoint =  'https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=60.10&lon=9.58'

# Issue an HTTP GET request
#r = requests.get(endpoint, auth=(client_id,''))
#, parameters, auth=(client_id,''))
# Extract JSON data


params = {
    #'sitename':'mahax@computas.com',
    'lat' : '59.94',
    'lon': '10.72',
    
}
user_agent = {
    "User-Agent": "computas.com mahax@computas.com"
}
#

#r = requests.get('https://api.met.no/weatherapi/locationforecast/2.0/compact.json?lat=59.9&lon=10.7')
r = requests.get('https://api.met.no/weatherapi/locationforecast/2.0/compact',params=params, headers=user_agent)

json = r.json()
json['properties']['timeseries'][1]['data']['instant']['details']['air_pressure_at_sea_level']

1015.1

In [36]:
dfForecast = pd.DataFrame()
data = json['properties']['timeseries']
for i in range(len(data)-2):

    row = pd.json_normalize(data[i]['data']['instant']['details'])

    row["datetime"] = [data[i]['time']]
    try:
        row['precipitation'] = [data[i]['data']['next_1_hours']['details']['precipitation_amount']]
    except:
        row['precipitation'] = [round(data[i]['data']['next_6_hours']['details']['precipitation_amount']/6,3)]
        

    dfForecast = dfForecast.append(row)
import pytz
#timezone = pytz.timezone("US/Central")
#timezone.localize(pd.to_datetime(dfForecast['datetime'][0],utc=True))

dfForecast['datetime']=np.array(dfForecast['datetime'],dtype='datetime64')
#dfForecast = dfForecast.reset_index(drop=True)
#dfForecast['datetime'].year
#.tz_convert('US/Central')
#from datetime import datetime as dt
#date=pd.to_datetime(dfForecast['datetime'][0])
#date.dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
dfForecast

,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,datetime,precipitation
0,1015.6,26.1,0.0,34.3,187.0,2.6,2021-07-22 12:00:00,0.0
0,1015.1,26.9,0.0,32.7,183.6,2.7,2021-07-22 13:00:00,0.0
0,1014.7,27.5,0.0,31.8,180.5,2.7,2021-07-22 14:00:00,0.0
0,1014.6,27.8,0.0,31.2,201.2,2.5,2021-07-22 15:00:00,0.0
0,1014.4,27.2,0.0,38.0,206.5,4.2,2021-07-22 16:00:00,0.0
...,...,...,...,...,...,...,...,...
0,1007.0,15.5,70.3,86.4,197.4,2.0,2021-07-30 06:00:00,0.0
0,1006.9,20.0,63.3,57.4,181.6,2.7,2021-07-30 12:00:00,0.0
0,1005.5,18.1,64.8,74.1,193.9,2.3,2021-07-30 18:00:00,0.0
0,1006.5,14.8,65.6,93.7,194.7,1.6,2021-07-31 00:00:00,0.0


# Luftkvalli

In [37]:
#json['properties']['timeseries']
#{fromtime}/{totime}/{latitude}/{longitude}/{radius}
params = {
    "fromtime": '2020-01-01T00:00:00+01:00',
    "totime": '2021-07-18T01:00:00+01:00',
    "latitude": '59.92',
    "longitude": '10.77',
    "radius" :'9'
}

r = requests.get('https://api.nilu.no/obs/historical/2019-01-01/2021-07-18/Hjortnes')
jsonHjortnes=r.json()
r = requests.get('https://api.nilu.no/obs/historical/2019-01-01/2021-07-18/Manglerud')
jsonManglerud = r.json()
r = requests.get('https://api.nilu.no/obs/historical/2019-01-01/2021-07-18/Smestad')
jsonSmestad = r.json()
r = requests.get('https://api.nilu.no/obs/historical/2019-01-01/2021-07-18/Kirkeveien')
jsonKirkeveien = r.json()
r = requests.get('https://api.nilu.no/obs/historical/2019-01-01/2021-07-18/E6%20Alna%20senter')
jsonE6AlnaSenter = r.json()
r = requests.get('https://api.nilu.no/obs/historical/2019-01-01/2021-07-18/Bygdøy%20Alle')
jsonBygdøyAlle = r.json()
r = requests.get('https://api.nilu.no/obs/historical/2019-01-01/2021-07-18/Bryn%20Skole')
jsonBrynSkole = r.json()


In [62]:
jsonBrynSkole

[{'id': 3041,
  'zone': 'Stor-Oslo',
  'municipality': 'Oslo',
  'area': 'Oslo',
  'station': 'Bryn skole',
  'type': 'Bybakgrunnsstasjon',
  'eoi': 'NO0132A',
  'component': 'NO',
  'latitude': 59.91416,
  'longitude': 10.82472,
  'timestep': 3600,
  'isVisible': False,
  'unit': 'µg/m³',
  'values': [{'fromTime': '2019-01-01T00:00:00+01:00',
    'toTime': '2019-01-01T01:00:00+01:00',
    'value': 2.058644,
    'qualityControlled': True},
   {'fromTime': '2019-01-01T01:00:00+01:00',
    'toTime': '2019-01-01T02:00:00+01:00',
    'value': 2.750515,
    'qualityControlled': True},
   {'fromTime': '2019-01-01T02:00:00+01:00',
    'toTime': '2019-01-01T03:00:00+01:00',
    'value': 0.834566,
    'qualityControlled': True},
   {'fromTime': '2019-01-01T03:00:00+01:00',
    'toTime': '2019-01-01T04:00:00+01:00',
    'value': 0.423508,
    'qualityControlled': True},
   {'fromTime': '2019-01-01T04:00:00+01:00',
    'toTime': '2019-01-01T05:00:00+01:00',
    'value': 0.30037,
    'qualityContr

In [52]:
df = pd.DataFrame()
data = [jsonHjortnes,jsonManglerud,jsonSmestad,jsonKirkeveien,jsonE6AlnaSenter,jsonBygdøyAlle,jsonBrynSkole]
from tqdm import tqdm
for name in tqdm(data):
    for i in range(len(name)):

        row = pd.json_normalize(name[i]['values'])
        row=row.drop(["toTime"],axis=1)

        row["station"] = [name[i]["station"]]*len(row)
        row["component"] = [name[i]["component"]]*len(row)
        row["type"] = [name[i]["type"]]*len(row)

        df = df.append(row)

    df = df.reset_index(drop=True)
 

100%|█████████████████████████████████████████████| 7/7 [00:02<00:00,  2.53it/s]


In [54]:
df.fromTime= [str(v) for v in np.array(df['fromTime'],dtype='datetime64')]
df['fromTime'][0]

'2018-12-31T23:00:00'

In [56]:
df.to_csv("data/luftkvalitetAggregertTime.csv",sep=';',index=False)

In [96]:
df.fromTime=pd.to_datetime(np.array(df['fromTime'],dtype='datetime64'))

In [55]:
df

,fromTime,value,qualityControlled,station,component,type
0,2018-12-31T23:00:00,58.410000,True,Hjortnes,PM10,Veinær stasjon
1,2019-01-01T00:00:00,130.460000,True,Hjortnes,PM10,Veinær stasjon
2,2019-01-01T01:00:00,7.590000,True,Hjortnes,PM10,Veinær stasjon
3,2019-01-01T02:00:00,0.550000,True,Hjortnes,PM10,Veinær stasjon
4,2019-01-01T03:00:00,3.300000,True,Hjortnes,PM10,Veinær stasjon
...,...,...,...,...,...,...
776749,2021-07-17T16:00:00,17.679463,False,Bryn skole,PM10,Bybakgrunnsstasjon
776750,2021-07-17T17:00:00,9.036547,False,Bryn skole,PM10,Bybakgrunnsstasjon
776751,2021-07-17T18:00:00,2.699494,False,Bryn skole,PM10,Bybakgrunnsstasjon
776752,2021-07-17T19:00:00,5.176111,False,Bryn skole,PM10,Bybakgrunnsstasjon
